# Perspektiva - měření výšky v obraze
Cvičení je zaměřené na vyzkoušení si měření výšky objektů v obraze pomocí úběžníků a úběžnic (viz přednáška [Perspektiva a geometrie obrazu](../../../lectures/files/bi-svz-05-perspektiva-obrazu.pdf)). 

Z pohledu zpracování obrazu se jedná o hranovou detekci pomocí [Cannyho detektoru](https://en.wikipedia.org/wiki/Canny_edge_detector) následovanou [Houghovou transformací](https://en.wikipedia.org/wiki/Hough_transform), která nalezené hrany aproximuje přímkami. K zisku úběžníků a úběžnic z nalezených přímek je využit algoritmus [RANSAC](https://en.wikipedia.org/wiki/Random_sample_consensus) převzatý z [githubu](https://github.com/chsasank/Image-Rectification). Ten vyhledá nejvhodnější přímky jdoucí stejným směrem postupně do všech tří úběžníků.

Následně je využit vlastní algoritmus pro odhad výšky objektů `HeightEstimator` na základě měření známých rozměrů v obraze. **Měřit je nutné vždy od spojnice roviny horizontální s rovinou vertikální ve směru osy z.**

Na obrázku příkladu je znázorněno měření dvou referenčních vzdáleností. Dále je na něm znázorněna reálná výška dvou objektů - osob. Měření se zadává vždy v pořadí: bod TL (top-left) a následně bod BR (bottom-right).

![](images/miry.png)

### Import knihoven a konfigurace

In [ ]:
%run ../svz.ipynb

from skimage import feature, color, transform

import warnings
warnings.filterwarnings('ignore')

### Pomocné funkce
Seznam funkcí pro přehlednost:
- [`show_images(...)`](../svz.ipynb#show_functions)


- [`Funkce a modely pro výpočty odhadu výšky`](../svz.ipynb#height_estimation_functions)

---

### Úkol
Vytvořte umělou scénu sestávající se ze dvou šachovnic na sebe kolmých. Do scény umístěte objekt, jehož výšku chcete změřit pomocí kamery.

Kamerou získejte obrázek a uložte ho. Následně změřte ručně v obraze dvě a více vzdáleností ve směru osy z (čím více vzdáleností, tím přesnější výsledek). Stejné vzdálenosti změřte i v reálné scéně.

Pomocí algoritmu získejte úběžnice a úběžníky. Pomocí nich spočtěte reálnou výšku objektu v obraze.

#### 1) Vytvořte umělou scénu a změřte reálné rozměry v ose z
Umělá scéna by měla vypadat nějak podobně jako na obrázku. 

<img src="images/artificial_scene_chess.png" width="50%">

#### 2) Získejte obrázek umělé scény kamerou Basler
Využijte jakýkoli nástroj, se kterým se již pracovalo (Pylon, [Práce s Basler kamerami notebook](../2/basler-introduction.ipynb)). Dejte pozor na ideální světelné podmínky, zaostření, clonu, expoziční čas, atd!

V případě, že záběr obrázku je větší než samotná scéna, ořízněte si obrázek tak, aby na něm okolí nebylo.

In [ ]:
image_path = '' ###
img = cv2.imread(image_path)

#### 3) Získejte souřadnice úběžníků

In [ ]:
# Funkce compute_vanishing_points vykresluje obrázky pomocí knihovny matplotlib,
# která očekává kanály v pořadí RGB a ne BGR jako OpenCV
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
vps = compute_vanishing_points(img_rgb)

#### 4) Naklikejte dvojice referenčních a měřených bodů v obraze
Vždy se ze dvojice nejdříve volí TL a následně až BR. Zvolte celkem 3 dvojice referenčních bodů (ty, pro které máte změřené reálné vzdálenosti). Referenční body se volí **vždy** v rovině vertikální (viz [úvod](#Perspektiva---měření-výšky-v-obraze)). Následně zvolte 1 dvojici bodů měřených.

Souřadnice se zobrazí jako výstup buňky.

In [ ]:
# Zobrazení obrázku pro klikání bodů
show_images(img)

#### 5) Souřadnice zkopírujte a doplňte na správná místa skriptu
`REFERENCE_MEASUREMENTS` jsou referenční dvojice bodů a jejich reálná vzdálenost. `TL` a `BR` je dvojice bodů měřených.

**POZOR:** Vypočítané úběžníky nemusí nutně být ve správném pořadí! Zkontrolujte, zda `vp1` a `vp2` jsou horizontální úběžníky a `vpz` je úběžník v ose z. V případě, že tomu tak není, zaměňte indexy u `vps` tak, aby tomu tak bylo.

In [ ]:
# Struktura: TL x,y; BR x,y; reálná vzdálenost
# např. ((10, 10), (20, 20), 100)
REFERENCE_MEASUREMENTS = [
    ((?, ?), (?, ?), ?), ###
    ((?, ?), (?, ?), ?), ###
    ((?, ?), (?, ?), ?) ###
]

# Dvojice měřených bodů objektů z obrazu
TL = (?, ?) ###
BR = (?, ?) ###

# POZOR! Úběžníky nemusí být ve správném pořadí!
vp1 = vps[0]
vp2 = vps[1]
vpz = vps[2]

#### 6) Využijte algoritmus pro výpočet odhadu výšky měřeného objektu

In [ ]:
# Horizont
vl = np.cross(vp1, vp2)

# Vytvoření objektu pro výpočet odhadu výšky objektu a zavolání funkce
height_estimator = HeightEstimator(REFERENCE_MEASUREMENTS, vl, vpz)
height = abs(height_estimator.calc_height(TL, BR))

# Výstup
print('Objekt {:.2f} mm'.format(height))

---
#### 7*) Dobrovolná úloha za 1 bod aktivity
Upravte výše uvedené tak, abyste vypočítali výšku lidí z obrazu `test.jpg`. Pro kontrolu je reálná výška zobrazena na obrázku výše.